# Gen.jl Problem Set

This notebook asks you to do some programming in Gen.jl, by filling in missing code blocks. The notebook asks you to:
- Make a small extension to the model program to make the speed of the drone a random variable
- Write a custom proposal to make inference in your extended model program more efficient
- Experiment with changes to the scene and/or dataset and their effect on the inferred destinations.
- Check the automated inferences against your common sense understanding.

In [ ]:
using Gen

In [ ]:
addprocs(4);

In [ ]:
@everywhere include("resources/goals/scene.jl")
@everywhere include("resources/goals/path_planner.jl")
@everywhere include("resources/goals/uniform_2d.jl");

In [ ]:
include("resources/goals/rendering.jl")

In [ ]:
@program agent_model() begin
    
    # assumed scene
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    
    wall_height = 30.
    walls = @e([
        Wall(Point(20., 40.), 1, 40., 2., wall_height),
        Wall(Point(60., 40.), 2, 40., 2., wall_height),
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height),
        Wall(Point(20., 80.), 1, 15., 2., wall_height),
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ], "walls")
    for wall in walls
        add!(scene, wall)
    end
    
    # time points at which we observe the agent's location (every 10 time steps)
    observation_times = @e(collect(linspace(0.0, 200.0, 20)), "times")
    
    # assumed speed of the agent
    speed = 1.0
    
    # the starting location of the agent is a random point in the scene
    start = @g(uniform_2d(0, 100, 0, 100), "start")
    
    # the destination of the agent is a random point in the scene
    destination = @g(uniform_2d(0, 100, 0, 100), "destination")
    
    # the path of the agent from its start location to its destination
    # uses a simple 2D holonomic path planner based on RRT (path_planner.jl)
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    if isnull(final_path)
        
        # the agent could not find a path to its destination
        # assume it stays at the start location indefinitely
        locations = [start for _ in observation_times]
    else
        
        # the agent found a path to its destination
        # assume it moves from the start to the destinatoin along the path at constnat speed
        # sample its location along this path for each time in observation times
        locations = walk_path(get(final_path), speed, observation_times)
    end
    
    # assume that the observed locations are noisy measurements of the true locations
    # assume the noise is normally distributed with standard deviation 'noise'
    noise = 1.0
    for (i, t) in enumerate(observation_times)
        measured_x = @g(normal(locations[i].x, noise), "x$i")
        measured_y = @g(normal(locations[i].y, noise), "y$i")
    end
    
    # record other program state for rendering
    @e(final_path, "final-path")
    @e(scene, "scene")
end;

# Problem 1

Modify the `agent_model` program above to make the speed of the drone a random variable. Show a number of traces sampled from the prior, and confirm that there is variability in the agent's speed between the simulations. You can use the starter code below to help generate the renderings. There are 2 rows and 6 columns in the tile plot. Sample 12 traces from your modified program and render them below.

It might be easier to see the effect of your change to the model if you fix the start and destination using the `constrain!` function.

NOTE: `Gen.gamma` conflicts with Julia's built-in `Base.gamma`; if you want to sample from a gamma distribution, use `Gen.gamma` not `gamma`.

In [ ]:
renderer = JupyterInlineRenderer("agent_model_renderer", Dict());
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
for i=1:12
    trace = ProgramTrace()
    
    # CODE HERE
    
    attach(renderer, id(figure => i))
    render(renderer, trace)
end

# Problem 2

In this problem, you will show that you can infer the speed that the drone is moving, and use the inferred speed to predict the future location of the drone.

First, create two test datasets of around 5 data points, that each have the drone moving at two different speeds. The speeds should be in the support of your model's prior distribution on the speed.

In [ ]:
dataset1 = [Point(10, 10)] # REPLACE ME

In [ ]:
dataset2 = [Point(10, 10)] # REPLACE ME

Next, fill in the function below to do generic importance sampling with the prior as the importance (i.e. proposal) distribution. The first argument is the number of importance samples to user internally, and the second argument is a sequence of initial observed locations of the drone.

We've started by constaining the start location of the drone in each importance sample to the first datapoint. Add the other necessary constraints. Use the other notebooks as a reference.

In [ ]:
function basic_inference(num_samples::Int, dataset::Vector)
    scores = Vector{Float64}(num_samples)
    traces = Vector{ProgramTrace}(num_samples)
    for k=1:num_samples
        trace = ProgramTrace()
        constrain!(trace, "start", dataset[1])
        
        # CODE HERE
        
        (scores[k], _) = @generate!(agent_model(), trace)
        traces[k] = trace
    end
    chosen = categorical_log(scores)
    return traces[chosen]
end;

Now, run the inference algorithm on the two datasets. For each dataset, run the inference algorithm 12 times to generate 12 independent approximate posterior samples, and show them in two separate tile plots.

Each trace that is returned by the inference algorithm will contain a complete set of locations of the drone across the entire time sequence. The prefix of this sequence will match the constrained values. The rest will be a prediction of the remainder of the drone's path. Each rendering will show both the constrained and predicted locations. Use the renderings to confirm that the inference algorithm is inferring the speed correctly, and that the inferred speed is taken into account in the predictions.

How many importance samples (the first argument to `basic_inference`) were necessary to get decent inferences about the speed?

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run inference on dataset1 and show 12 samples in the output of the cell above

# CODE HERE

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run inference on dataset2 and show 12 samples in the output of the cell above

# CODE HERE

# Problem 3

The `basic_inference` algorithm above guesses the speed by sampling it from the prior when generating each importance sample. For datasets where the drone is moving at roughly a constant speed, the speed should be pretty obvious from the dataset, and its not necessary to guess it from the prior. Fill in the probabilistic program `custom_proposal` below with code that makes an intelligent guess about the speed, given some prefix of observed locations of the drone. We've put a line at the end that samples from a standard normal disribution, and annotates that sample with the address "speed". Change the function so that "speed" is sampled from a distribution that depends on the dataset.

Remember that the best proposal distribution is the posterior, and the posterior probably has some variability. You want your proposal to have some variability, but not too much (because otherwise the knowledge encoded in the program isn't being used). Experiment a bit with the parameters in your function and observe the effect on the quality of inferences.

In [ ]:
custom_proposal = @program (dataset::Vector) begin
    
    # CODE HERE

    @g(normal(0, 1), "speed") # REPLACE ME
end

Below, we have written a function `custom_inference` that uses your custom proposal in an importance sampler. You don't need to modify this function.

In [ ]:
function custom_inference(num_samples::Int, dataset::Vector)
    
    # create a new generator that samples from a custom importance distribution by 
    # sampling the 'speed' from the custom_proposal program, and the remaining random choices
    # from the prior, and returns the log importance weight as its score during `generate!`
    composition = compose(agent_model, custom_proposal, Dict(["speed" => ("speed", Float64)]))
    
    trace = ProgramTrace()
    constrain!(trace, "start", dataset[1])
    for (i, point) in enumerate(dataset)
        constrain!(trace, "x$i", point.x)
        constrain!(trace, "y$i", point.y)
    end
    traces = Vector{ProgramTrace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        (scores[k], _) = @generate!(composition((), (dataset,)), t)
        traces[k] = t
    end
    chosen = categorical_log(scores)
    return traces[chosen]
end;

Render samples from `custom_inference` for the two datasets as you did above for `basic_inference`. Can you get  reasonable predictions using fewer importance samples (and therefore less computation) than you did with `basic_inference`?

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run inference on dataset1 and show 12 samples in the output of the cell above

# CODE HERE

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run inference on dataset2 and show 12 samples in the output of the cell above

# CODE HERE

# Aside: Nested Inference

Next, we introduce the concept of "nested inference", in which parts of a probabilistic model come with their own inference code built into them. This is a capability enabled by Gen's flexible `Generator` interface. Allowing modeling code to have built-in inference code for its internal random variables can allow for more modular inference programming. We'll demonstrate this below.

Suppose that what we really care about is predicting the drone's future locations, and its destination. The speed of the drone is not of direct domain interest---it is only important to model because it is relevant to inferences about the variables we do care about directly. We can try to follow good software engineering principles, and break our large monolithic `agent_model` procedure into smaller individually coherent units. Let's separate the parts of the model concerned with the motion of the drone from the parts concerned with the plan, by creating a separate `motion_model` procedure.

Please copy your modified distribution on `speed` into the code below:

In [ ]:
@program motion_model(start::Point, final_path::Nullable{Path}, observation_times::Vector{Float64}) begin
    
    speed = 1.0 # REPLACE ME
    
    if isnull(final_path)
        locations = [start for _ in observation_times]
    else
        locations = walk_path(get(final_path), speed, observation_times)
    end
    noise = 1.0
    measured = Vector{Point}(length(observation_times))
    for (i, t) in enumerate(observation_times)
        measured_x = @g(normal(locations[i].x, noise), "x$i")
        measured_y = @g(normal(locations[i].y, noise), "y$i")
        measured[i] = Point(measured_x, measured_y)
    end
    measured
end

Now, the new top-level model program is shorter. We add "address aliases" so that we can still use the addresses `"speed"`, `"x$i"` and `"y$i"` to refer to these random variables. If we didn't have the address aliases, the addresses would be tuples: `("motion", "speed")`, `("motion", "x$i")`, `("motion", "y$i")`. In general, we can address parts of a program's execution deep within a call stack of probabilistic programs---the addresses just become long tuples.

In [ ]:
@program agent_model_refactored() begin
    
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    walls = @e([
        Wall(Point(20., 40.), 1, 40., 2., wall_height),
        Wall(Point(60., 40.), 2, 40., 2., wall_height),
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height),
        Wall(Point(20., 80.), 1, 15., 2., wall_height),
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ], "walls")
    for wall in walls
        add!(scene, wall)
    end

    observation_times = @e(collect(linspace(0.0, 200.0, 20)), "times")
    start = @g(uniform_2d(0, 100, 0, 100), "start")
    destination = @g(uniform_2d(0, 100, 0, 100), "destination")
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    @alias("speed", ("motion", "speed"))
    for i=1:length(observation_times)
        @alias("x$i", ("motion", "x$i"))
        @alias("y$i", ("motion", "y$i"))
    end
    measured_locations = @g(motion_model(start, final_path, observation_times), "motion")

    # record other program state for rendering
    @e(final_path, "final-path")
    @e(scene, "scene")
end;

Let's confirm that inference still works by running on `dataset1` again. Let's copy the inference procedure and make it run `agent_model_refactored` instead of `agent_model`:

In [ ]:
function custom_inference_refactored(num_samples::Int, dataset::Vector)

    composition = compose(agent_model_refactored, custom_proposal, Dict(["speed" => ("speed", Float64)]))
    
    trace = ProgramTrace()
    constrain!(trace, "start", dataset[1])
    for (i, point) in enumerate(dataset)
        constrain!(trace, "x$i", point.x)
        constrain!(trace, "y$i", point.y)
    end
    traces = Vector{ProgramTrace}(num_samples)
    scores = Vector{Float64}(num_samples)
    for k=1:num_samples
        t = deepcopy(trace)
        (scores[k], _) = @generate!(composition((), (dataset,)), t)
        traces[k] = t
    end
    chosen = categorical_log(scores)
    return traces[chosen]
end;

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run inference on dataset1 and show 12 samples in the output of the cell above

# CODE HERE

So far, we have refactored our model program, but our inference program `custom_inference_refactored` still includes details about how to infer the speed. We can also achieve more modularity in the inference program using **nested inference**. Specifically, we implement a custom `Generator` type for the motion model, that will include nested inference over its internal "speed" random choice. 

Recall that a probabilistic program in Gen (as created with `@program`) is one type of `Generator`. A probabilistic program describes the forward generative process, but doesn't let the programmer define inference algorithms for that program within the program itself. The more general `Generator` interface is flexible enough to permit custom inference that is packaged together with the description of the generative process. We'll call our new `Generator` type `MotionModelGenerator`. We have to implement the `empty_trace` method, which returns an empty trace that this generator can write to, and the `generate!` method, which is the main entry-point into the generator:

In [ ]:
struct MotionModelGenerator <: Generator{ProgramTrace}
end

In [ ]:
Gen.empty_trace(g::MotionModelGenerator) = ProgramTrace()

We'll write our generator so that it can generate into traces that have a consecutive block of measurements constrained. In that case, it will sample the speed from the custom speed proposal distribution, and return the importance weight as its score. If there are no constraints on the measurements, it behaves just like a probabilistic program generator.

In [ ]:
function Gen.generate!(generator::MotionModelGenerator, args::Tuple, trace::ProgramTrace)
    start, path, times = args
    num_points = length(times)
    constrained = false
    dataset = []
    done = false
    constrained = false
    for i=1:num_points
        x_constrained = haskey(trace, "x$i") && Gen.mode(trace, "x$i") == Gen.constrain
        y_constrained = haskey(trace, "y$i") && Gen.mode(trace, "y$i") == Gen.constrain
        if !x_constrained && !y_constrained
            done = true
        elseif x_constrained && y_constrained
            constrained = true
            if done
                error("generator only supports constraints a consecutive prefix of points")
            end
            push!(dataset, Point(trace["x$i"], trace["y$i"]))
        else
            error("x and y must both be constrained or both not constrained")
        end
    end
    if constrained
        
        # nested inference using the custom proposal to 'speed'
        # generate! returns an importance weight
        generator = compose(motion_model, custom_proposal, Dict(["speed" => ("speed", Float64)]))
        return Gen.generate!(generator, (args, (dataset,)), trace)
    else
        
        # behave like a probabilistic program
        return Gen.generate!(motion_model, args, trace)
    end
end

We modify the agent model again, to use our new custom `MotionModelGenerator`:

In [ ]:
@program agent_model_nested_inference() begin
    
    scene = Scene(0, 100, 0, 100) # the scene spans the square [0, 100] x [0, 100]
    add!(scene, Tree(Point(30, 20))) # place a tree at x=30, y=20
    add!(scene, Tree(Point(83, 80)))
    add!(scene, Tree(Point(80, 40)))
    wall_height = 30.
    walls = @e([
        Wall(Point(20., 40.), 1, 40., 2., wall_height),
        Wall(Point(60., 40.), 2, 40., 2., wall_height),
        Wall(Point(60.-15., 80.), 1, 15. + 2., 2., wall_height),
        Wall(Point(20., 80.), 1, 15., 2., wall_height),
        Wall(Point(20., 40.), 2, 40., 2., wall_height) ], "walls")
    for wall in walls
        add!(scene, wall)
    end

    observation_times = @e(collect(linspace(0.0, 200.0, 20)), "times")
    start = @g(uniform_2d(0, 100, 0, 100), "start")
    destination = @g(uniform_2d(0, 100, 0, 100), "destination")
    (tree, rough_path, final_path) = plan_path(start, destination, scene)
    
    for i=1:length(observation_times)
        @alias("x$i", ("motion", "x$i"))
        @alias("y$i", ("motion", "y$i"))
    end
    measured_locations = @g(MotionModelGenerator()(start, final_path, observation_times), "motion")

    # record other program state for rendering
    @e(final_path, "final-path")
    @e(scene, "scene")
end;

Let's sample from the new generative program to make sure it behaves just likethe original one:

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
for i=1:12
    trace = ProgramTrace()
    constrain!(trace, "start", Point(10, 10))
    constrain!(trace, "destination", Point(90, 90))
    @generate!(agent_model_nested_inference(), trace)
    attach(renderer, id(figure => i))
    render(renderer, trace)
end

Now, we write an importance sampling program for use with the new `agent_model_nested_inference` model program. Crucially, this inference program is identical to our original `basic_inference` program---it doesn't contain mention of the speed, or of our custom proposal. Instead, the custom proposal is being invoked under the hood within the `MotionModelGenerator` every time we call `generate!` on the model program. The importance distribution used by `using_nested_inference` is the same as the importance distribution used in `custom_inference`, except that now the custom inference over the parameter of the motion model (the speed has been encapsulated inside the motion model itself:

In [ ]:
function using_nested_inference(num_samples::Int, dataset::Vector)
    scores = Vector{Float64}(num_samples)
    traces = Vector{ProgramTrace}(num_samples)
    for k=1:num_samples
        trace = ProgramTrace()
        constrain!(trace, "start", dataset[1])
        for (i, point) in enumerate(dataset)
            constrain!(trace, "x$i", point.x)
            constrain!(trace, "y$i", point.y)
        end
        (scores[k], _) = @generate!(agent_model_nested_inference(), trace)
        traces[k] = trace
    end
    chosen = categorical_log(scores)
    return traces[chosen]
end;

Let's verify that the inference algorithm still works by testing it on `dataset1` and `dataset2`:

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run using_nested_inference on dataset1 and show 12 samples in the output of the cell above

# CODE HERE

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
# Run using_nested_inference on dataset2 and show 12 samples in the output of the cell above

# CODE HERE

# Problem 4

Recall that we can modify the scene by using `intervene!` on the "walls" as done in the earlier notebook, e.g.:

    intervene!(trace, "walls", new_walls)

Recall that `intervene!` is like `constrain!` in that it fixes the given address to a particular value in the execution of the program. However, interventions are not incorporated into the score returned by `generate!`. Interventions simply replace the value at an address. Unlike constraints, which can only be applied to generator invocations annotated with `@g`, interventions can be applied to generator invocations or arbitrary expressions annotated with `@e`. In this case, the "walls" are not generated by a generator invocation, and we are simply replacing that expression in the program with a new value. Note that the modification of the scene can also be achieved by making the scene an argument.

By changing the dataset and/or scene, show that a change to the scene can result in drastic changes to the distribution on predicted paths, and approximate posterior distribution on destinations.

We've written another inference program that takes a trace called `base_trace` as input. This `base_trace` can contain interventions, which will be copied into all of the importance samples used within the inference algorithm.

In [ ]:
function inference(num_samples::Int, dataset::Vector, base_trace::ProgramTrace)
    scores = Vector{Float64}(num_samples)
    traces = Vector{ProgramTrace}(num_samples)
    for k=1:num_samples
        trace = deepcopy(base_trace)
        constrain!(trace, "start", dataset[1])
        for (i, point) in enumerate(dataset)
            constrain!(trace, "x$i", point.x)
            constrain!(trace, "y$i", point.y)
        end
        (scores[k], _) = @generate!(agent_model_nested_inference(), trace)
        traces[k] = trace
    end
    chosen = categorical_log(scores)
    return traces[chosen]
end;

In [ ]:
dataset = [Point(10, 10)] # REPLACE ME

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)

here(figure)

For reference, the `Wall` constructor has type signature:

    Wall(start::Point, orientation::Int, length::Float64, thickness::Float64, height::Float64)

`orientation=1` indicates a horizontal wall, and an `orientation=2` indicates a vertical wall.

Note that the `height` does not matter, because the drone planner is operating in 2D space.

In [ ]:
base_trace = ProgramTrace()
intervene!(base_trace, "walls", []) # REPLACE ME
for i=1:20
    attach(renderer, id(figure => i))
    trace = inference(20, dataset, base_trace)
    render(renderer, trace)
end

In [ ]:
figure = Figure(num_rows=2, num_cols=6, width=900, height=300, trace_width=100, trace_height=100)
here(figure)

In [ ]:
base_trace = ProgramTrace()
intervene!(base_trace, "walls", []) # REPLACE ME
for i=1:20
    attach(renderer, id(figure => i))
    trace = inference(20, dataset, base_trace)
    render(renderer, trace)
end

# Problem 5

Experiment! Change the model and/or dataset and/or inference programs in other ways.

Some concrete suggestions:

- Try making some properties of the scene into random choices. Can you infer something about the drone's beliefs about the scene from its motion? Do the automated inferences match your common sense inferences? Can you set up a scene and dataset to clearly illustrate that inference is working in your model? 

- Try improving the flexibility of the motion model further by making the noise a random variable. Test it on a dataset that matches the model's expectations and one that doesn't. Can you demonstrate that your model in which the noise level is inferred, is more robust to model mis-specification?